In [ ]:
import tensorcircuit as tc
import jax.numpy as np
import optax
from tensorcircuit . experimental import qng
from functools import partial

In [ ]:
backend = tc.set_backend("jax")
tc.set_dtype("complex128")
tc.set_contractor("auto")  # “auto”, “greedy”, “branch”, “plain”, “tng”, “custom”

In [ ]:
n = 4
k = 3
#
params = backend.implicit_randn([3 * n, k + 1], dtype="complex")

def sensor(params, phi):
    p = 0.0
    dmc = tc.Circuit(n)

    params_probe = params[:, 0:-1]
    params_measure = params[:, -1:]

    # probe state
    # for layer in range(k):
    #     for i in range(n):
    #         dmc.r(i, theta=params_probe[i * 3, k],
    #               alpha=params_probe[i])
    #     for i in range(1, n):
    #         dmc.cnot(i-1, i)
    #     for i in range(n):
    #         dmc.depolarizing(i, px=p, py=p, pz=p)
    dmc.h(0)
    for i in range(1, n):
        dmc.cnot(0, i)

    # interaction
    for i in range(n):
        dmc.rz(i, theta = phi)

    # measurement
    # for i in range(n):
    #     dmc.rx(i, theta=params_measure[i * 3, 0])
    #     dmc.ry(i, theta=params_measure[i * 3 + 1, 0])
    #     dmc.rz(i, theta=params_measure[i * 3 + 1, 0])

    return dmc.densitymatrix()


def cfi(params, phi):

    def prob(params, phi):
        dm = sensor(params, phi)
        return backend.real(backend.diagonal(dm))

    p = prob(params, phi)
    # dprob = backend.jit(backend.jacrev(lambda phi: prob(params=params, phi=phi)))
    dprob = backend.jacrev(lambda phi: prob(params=params, phi=phi))
    print(dprob(phi))
    print(dprob(phi).squeeze())
    fi = backend.sum((dprob(phi).squeeze()) ** 2 / p)
    return fi


# phi = np.array([1.12314])
phi = backend.implicit_randn()
print(cfi(params, phi))

In [ ]:
n = 4
k = 3
#
params = backend.implicit_randn([3 * n, k + 1])

def sensor(params, phi):
    p = 0.0
    # dmc = tc.Circuit(n)
    dmc = tc.DMCircuit(n)

    params_probe = params[:, 0:-1]
    params_measure = params[:, -1:]

    # probe state
    for layer in range(k):
        for i in range(n):
            dmc.r(i,
                  theta=params_probe[3*i, k],
                  alpha=params_probe[3*i + 1, k],
                  phi = params_probe[3*i + 2, k])
        for i in range(1, n):
            dmc.cnot(0, i)
    #     for i in range(n):
    #         dmc.depolarizing(i, px=p, py=p, pz=p)

    # dmc.h(0)
    # for i in range(1, n):
    #     dmc.cnot(0, i)

    # interaction
    for i in range(n):
        dmc.rx(i, theta = phi)

    # measurement
    for i in range(n):
        dmc.r(i,
              theta=params_measure[3*i, 0],
              alpha=params_measure[3*i + 1, 0],
              phi = params_measure[3*i + 2, 0])

    #     dmc.rx(i, theta=params_measure[i * 3, 0])
    #     dmc.ry(i, theta=params_measure[i * 3 + 1, 0])
    #     dmc.rz(i, theta=params_measure[i * 3 + 1, 0])

    return dmc.densitymatrix()
    # return dmc.wavefunction()


def cfi(params, phi):

    def prob(params, phi):
        dm = sensor(params, phi)
        return backend.real(backend.diagonal(dm))
        # return backend.real(dm)

    p = prob(params, phi)
    # dprob = backend.jit(backend.jacrev(lambda phi: prob(params=params, phi=phi)))
    dprob = backend.jacrev(lambda phi: prob(params=params, phi=phi))
    # print(dprob(phi))
    # print(dprob(phi).squeeze())
    fi = backend.sum((dprob(phi).squeeze()) ** 2 / p)
    return fi

print(backend.sum(backend.abs(sensor(params, phi))**2))
# phi = np.array([1.12314])
# phi = backend.implicit_randn()
# print(cfi(params, phi))

In [ ]:
cfi_val_grad_jit = backend.jit(backend.value_and_grad(lambda params: -cfi(params=params, _phi=phi)))
val, grad = cfi_val_grad_jit(params)

In [ ]:
opt = tc.backend.optimizer(optax.adagrad(learning_rate=0.95))


In [ ]:
params = backend.implicit_randn([3 * n, k + 1])

for i in range(250):
    val, grad = cfi_val_grad_jit(params)
    params = opt.update(grad, params)
    print(f"Step {i} | CFI {val}")
    # print(params)

In [ ]:
# %timeit noisy(4)

In [ ]:
n = 6
k = 4
#
params = backend.implicit_randn([3 * n, k]).astype("complex")


def sensor(params, phi):
    dmc = tc.Circuit(n)

    for i in range(k):
        for j in range(n):
            dmc.r(j, theta=params[3*j, i], alpha=params[3*j + 1, i], phi=params[3*j + 2, i])

        for j in range(1, n):
            dmc.cnot(j-1, j)

    # interaction
    for j in range(n):
        dmc.rz(j, theta = phi)
    return dmc

phi = np.array([1.12314]).astype("complex")
params = backend.implicit_randn([3 * n, k])

dmc = sensor(params, phi)
dmc.draw(output="text")


def qfi(_params, phi):
    psi = sensor(_params, phi).state()[:, None]
    f_dpsi_phi = backend.jacrev(lambda phi: sensor(params=_params, phi=phi).state())
    d_psi = f_dpsi_phi(phi)
    fi = 4 * backend.real((backend.conj(d_psi.T) @ d_psi) + (backend.conj(d_psi.T) @ psi)**2)
    return fi[0, 0]


In [ ]:
dmc.draw(output="text")

In [ ]:
qfi_val_grad_jit = backend.jit(backend.value_and_grad(lambda params: -qfi(_params=params, phi=phi)))
val, grad = qfi_val_grad_jit(params)
print(val, grad)

In [ ]:
opt = tc.backend.optimizer(optax.adagrad(learning_rate=0.35))

params = backend.implicit_randn([3 * n, k])

for i in range(250):
    val, grad = qfi_val_grad_jit(params)
    # print(grad)
    params = opt.update(grad, params)
    print(f"Step {i} | QFI {val}")
    # print(params)

# CFI

In [ ]:
n = 6
k = 4

def sensor(params, phi):
    dmc = tc.Circuit(n)

    for i in range(k):
        for j in range(n):
            dmc.r(j, theta=params[3*j, i], alpha=params[3*j + 1, i], phi=params[3*j + 2, i])

        for j in range(1, n):
            dmc.cnot(j-1, j)

    for j in range(n):
        dmc.r(j, theta=params[3*j, i], alpha=params[3*j + 1, i], phi=params[3*j + 2, i])

    # interaction
    for j in range(n):
        dmc.rz(j, theta = phi)

    # measurement
    for j in range(n):
        dmc.u(j, theta = params[3*j, -1], phi=params[3*j + 1, -1])

    return dmc


phi = np.array([1.12314])
params = backend.implicit_randn([3 * n, k+1])

dmc = sensor(params, phi)

def cfi(_params, _phi):
    def probs(_params, _phi):
        return backend.abs(sensor(_params, _phi).state())**2
    pr = probs(_params, _phi)
    dpr_phi = backend.jacrev(lambda _phi: probs(_params=_params, _phi=_phi))
    d_pr = dpr_phi(phi).squeeze()
    fim = backend.sum(d_pr * d_pr / pr)
    return fim

print(cfi(params, phi))


In [ ]:
dmc.draw(output="text")

In [ ]:
cfi_val_grad_jit = backend.jit(backend.value_and_grad(lambda params: -cfi(_params=params, _phi=phi)))
val, grad = cfi_val_grad_jit(params)
print(val, grad)

In [ ]:
opt = tc.backend.optimizer(optax.adagrad(learning_rate=0.2))
params = backend.implicit_randn([3 * n, k+1])

for i in range(250):
    val, grad = cfi_val_grad_jit(params)
    # print(grad)
    params = opt.update(grad, params)
    print(f"Step {i} | CFI {val}")
    # print(params)

# Mixed CFI

In [ ]:
n = 4
k = 4


def sensor(params, phi, gamma):
    dmc = tc.DMCircuit(n)

    for i in range(k):
        for j in range(n):
            dmc.r(j, theta=params[3 * j, i], alpha=params[3 * j + 1, i], phi=params[3 * j + 2, i])

        for j in range(1, n):
            dmc.cnot(j - 1, j)

        for j in range(n):
            dmc.phasedamping(j, gamma=gamma[0])

    for j in range(n):
        dmc.r(j, theta=params[3 * j, i], alpha=params[3 * j + 1, i], phi=params[3 * j + 2, i])

    # interaction
    for j in range(n):
        dmc.rz(j, theta=phi[0])

    # measurement
    for j in range(n):
        dmc.u(j, theta=params[3 * j, -1], phi=params[3 * j + 1, -1])

    return dmc


phi = np.array([1.12314])
gamma = np.array([0.0])
params = backend.implicit_randn([3 * n, k + 1])

dmc = sensor(params, phi, gamma)


def cfi(_params, _phi, _gamma):
    def probs(_params, _phi, _gamma):
        return backend.abs(backend.diagonal(sensor(_params, _phi, _gamma).densitymatrix()))

    pr = probs(_params, _phi, _gamma)
    dpr_phi = backend.jacrev(lambda _phi: probs(_params=_params, _phi=_phi, _gamma=_gamma))
    d_pr = dpr_phi(phi).squeeze()
    fim = backend.sum(d_pr * d_pr / pr)
    return fim

print(cfi(params, phi, gamma))

def neg_cfi(_params, _phi, _gamma):
    return -cfi(_params, _phi, _gamma)

In [ ]:
dmc.draw(output="text")

In [ ]:
cfi_val_grad_jit = backend.jit(backend.value_and_grad(lambda params: -cfi(_params=params, _phi=phi)))

# val, grad = cfi_val_grad_jit(params)
# print(val, grad)

In [ ]:
opt = tc.backend.optimizer(optax.adagrad(learning_rate=0.2))
params = backend.implicit_randn([3 * n, k + 1])

for i in range(250):
    val, grad = cfi_val_grad_jit(params)
    # print(grad)
    params = opt.update(grad, params)
    print(f"Step {i} | CFI {val}")
    # print(params)

In [ ]:
def optimal_information_under_dephasing(gamma):



for gamma in np.linspace(0, 1, 10):


In [ ]:
import jax

def func(a, b):
    return a*a*b


jax.jacrev(func, argnums=(0,))(0.1, 1.0)


In [ ]:
params = backend.implicit_randn([3])
def func(a, b, c):
    dmc = tc.Circuit(1)
    dmc.r(0, theta=a, alpha=b, phi=c)
    return dmc.state()
df = backend.jacrev(func, [0])
print(df(1.0, 1.0, 1.0))